<a href="https://colab.research.google.com/github/ashkanb77/Persian-Text-Summarization-Using-T5/blob/main/T5Persian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q transformers==4.5.0

     |████████████████████████████████| 2.1 MB 36.2 MB/s 
     |████████████████████████████████| 3.3 MB 69.6 MB/s 
     |████████████████████████████████| 880 kB 50.6 MB/s 


In [2]:
! pip install -q pytorch-lightning

     |████████████████████████████████| 585 kB 5.7 MB/s 
     |████████████████████████████████| 419 kB 48.8 MB/s 
     |████████████████████████████████| 140 kB 46.1 MB/s 
     |████████████████████████████████| 596 kB 9.8 MB/s 
     |████████████████████████████████| 1.1 MB 44.8 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 144 kB 19.5 MB/s 
     |████████████████████████████████| 271 kB 57.9 MB/s 


In [3]:
! pip install -q sentencepiece

     |████████████████████████████████| 1.2 MB 28.5 MB/s 


In [ ]:
import json
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pathlib import Path
from sklearn.model_selection import train_test_split
import textwrap
import  re

from transformers import AdamW, T5ForConditionalGeneration, T5Tokenizer, AutoTokenizer
from tqdm.auto import tqdm

In [ ]:
data_train = pd.read_csv('drive/MyDrive/datasets/text_summarization/train.csv', encoding='latin-1', on_bad_lines='skip')
data_test = pd.read_csv('drive/MyDrive/datasets/text_summarization/test.csv', encoding='latin-1', on_bad_lines='skip')

In [ ]:
data_train[['title', 'article', 'highlights']] = data_train[['title', 'article', 'highlights']].applymap(lambda x: str(x).encode('iso-8859-1').decode('utf-8', errors='ignore')).values
data_test[['title', 'article', 'highlights']] = data_test[['title', 'article', 'highlights']].applymap(lambda x: str(x).encode('iso-8859-1').decode('utf-8', errors='ignore')).values

In [ ]:
def clean(s):
  pattern = r'[آا-ی۱۲۳۴۵۶۷۸۹۰.!:()،؟]+'
  try:
     return ' '.join(re.findall(pattern, s))
  except:
    return s

In [ ]:
data_train[['title', 'article', 'highlights']] = data_train[['title', 'article', 'highlights']].applymap(lambda x: clean(x)).values
data_test[['title', 'article', 'highlights']] = data_test[['title', 'article', 'highlights']].applymap(lambda x: clean(x)).values

In [ ]:
class NewsDataset(Dataset):
  def __init__(self, data, tokenizer, text_max_len, summary_max_len):
    self.data = data
    self.tokenizer = tokenizer
    self.text_max_len = text_max_len
    self.summary_max_len = summary_max_len

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    data_row = self.data.iloc[index]

    text = data_row['article']

    text_encoding = self.tokenizer(
        text, max_length=self.text_max_len,
        padding='max_length', truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )

    summary_encoding = self.tokenizer(
        data_row['highlights'], max_length=self.summary_max_len,
        padding='max_length', truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )

    labels = summary_encoding['input_ids']
    labels[labels == 0] = -100

    return {
        'text':text,
        'summary':data_row['highlights'],
        'text_input_ids': text_encoding['input_ids'].flatten(),
        'text_attention_mask': text_encoding['attention_mask'].flatten(),
        'labels': labels.flatten(),
        'labels_attention_mask': summary_encoding['attention_mask'].flatten()
    }



In [ ]:
class NewsDataModule(pl.LightningDataModule):

  def __init__(self, train_df, test_df, tokenizer, batch_size, text_max_len, summary_max_len):
    super().__init__()
    self.train_df = train_df
    self.test_df = test_df
    self.tokenizer = tokenizer
    self.batch_size = batch_size
    self.text_max_len = text_max_len
    self.summary_max_len = summary_max_len

  def setup(self, stage=None):
    self.train_dataset = NewsDataset(
        self.train_df, self.tokenizer,
        self.text_max_len, self.summary_max_len
    )

    self.test_dataset = NewsDataset(
        self.train_df, self.tokenizer,
        self.text_max_len, self.summary_max_len
    )

  def train_dataloader(self):
    return DataLoader(
        self.train_dataset, self.batch_size,
        shuffle=True, num_workers=2
        )
    
  def test_dataloader(self):
    return DataLoader(
        self.test_dataset, self.batch_size,
        shuffle=True, num_workers=2
        )
    
  def val_dataloader(self):
    return DataLoader(
        self.test_dataset, self.batch_size,
        shuffle=True, num_workers=2
    )

In [ ]:
model_name = 't5-small'

tokenizer = AutoTokenizer.from_pretrained('Ahmad/parsT5-base')

Downloading:   0%|          | 0.00/678 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

In [ ]:
n_epochs = 4
batch_size = 8

data_module = NewsDataModule(data_train, data_test, tokenizer, batch_size, 500, 200)

In [ ]:
class NewsModel(pl.LightningModule):

  def __init__(self):
    super().__init__()
    self.model = T5ForConditionalGeneration.from_pretrained(model_name, return_dict=True)

  def forward(self, input_ids, attention_mask, decoder_attention_mask, labels=None):
    output = self.model(
        input_ids, attention_mask=attention_mask,
        labels=labels,
        decoder_attention_mask=decoder_attention_mask
    )

    return output.loss, output.logits

  def training_step(self, batch, batch_idx):
    input_ids = batch['text_input_ids']
    attention_mask = batch['text_attention_mask']
    labels = batch['labels']
    labels_attention_mask = batch['labels_attention_mask']

    loss, outputs = self(
        input_ids=input_ids,
        attention_mask=attention_mask,
        decoder_attention_mask=labels_attention_mask,
        labels=labels
    )

    return loss

  def validation_step(self, batch, batch_idx):
    input_ids = batch['text_input_ids']
    attention_mask = batch['text_attention_mask']
    labels = batch['labels']
    labels_attention_mask = batch['labels_attention_mask']

    loss, outputs = self(
        input_ids=input_ids,
        attention_mask=attention_mask,
        decoder_attention_mask=labels_attention_mask,
        labels=labels
    )

    return loss

  def configure_optimizers(self):
      return AdamW(self.parameters(), lr=0.0001)

In [ ]:
model = NewsModel()

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
checkpoint = ModelCheckpoint(
    dirpath='drive/MyDrive/checkpoints',
    filename='chp.ckp',
    save_top_k=1,
    verbose=True,
    monitor='val_loss',
    mode='min'
)

In [ ]:
trainer = pl.Trainer(
    max_epochs=n_epochs,
    gpus=1,
    checkpoint_callback=checkpoint,
    progress_bar_refresh_rate=30
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f65cdc64c90>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f65cdc64c90>)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=30)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Tr

In [ ]:
trainer.fit(model, data_module)

Missing logger folder: /content/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:490: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
def summarize(text):
  text_encoding = tokenizer(
      text,
      max_length=512,
      padding='max_length',
      truncation=True,
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors='pt'
  )

  generated_ids = trained_model.model.generate(
      input_ids=text_encoding['input_ids'],
      attention_mask=text_encoding['attention_mask'],
      max_length=150,
      num_beams=2,
      repetition_penalty=2.5,
      length_penalty=1.0,
      early_stopping=True
  )

  preds = [
           tokenizer.decode(gen_id, skip_special_tokens=True,
                            clean_up_tokenization_spaces=True) for gen_id in generated_ids
  ]

  return "".join(preds)

In [ ]:
# trained_model = NewsModel.load_from_checkpoint(
#     trainer.checkpoint_callback.best_model_path
# )

In [ ]:
trained_model = NewsModel.load_from_checkpoint(checkpoint_path="drive/MyDrive/checkpoints/chp.ckpt")

In [ ]:
trained_model.freeze()

In [ ]:
data_train.iloc[0]['article']

'به گزارش شانا، علی کاردر امروز (۲۷ دی ماه) در مراسم تودیع محسن قمصری، مدیر سابق امور بین الملل شرکت ملی نفت ایران و معارفه سعید خوشرو، مدیر جدید امور بین الملل این شرکت، گفت: مدیریت امور بین الملل به عنوان یکی از تاثیرگذارترین مدیریت های شرکت ملی نفت ایران در دوران تحریم های ظالمانه غرب علیه کشورمان بسیار هوشمندانه عمل کرد و ما توانستیم به خوبی از عهده تحریم ها برآییم. وی افزود: مجموعه امور بین الملل در همه دوران ها با سختی ها و مشکلات بسیاری مواجه بوده است، به ویژه در دوره اخیر به دلیل مسا ل پیرامون تحریم وظیفه سنگینی بر عهده داشت که با تدبیر مدیریت خوب این مجموعه سربلند از آن بیرون آمد. کاردر با قدردانی از زحمات محسن قمصری، به سلامت مدیریت امور بین الملل این شرکت اشاره کرد و افزود: محوریت کار مدیریت اموربین الملل سلامت مالی بوده است. وی بر ضرورت نهادینه سازی جوانگرایی در مدیریت شرکت ملی نفت ایران تاکید کرد و گفت: مدیریت امور بین الملل در پرورش نیروهای زبده و کارآزموده آنچنان قوی عملکرده است که برای انتخاب مدیر جدید مشکلی وجود نداشت. کاردر، حرفه ای گری و کار استاندارد را از ویژگی های

In [ ]:
summarize(data_train.iloc[0]['article'])

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1777: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


'مدیر سابق امور بین الملل شرکت ملی نفت ایران گفت: مدیریت امور بین الملل به عنوان یکی از تاثیرگذارترین مدیریت های شرکت ملی نفت ایران در دوران تحریم های ظالمانه غرب علیه کشورمان بسیار هوشمندانه عمل کرد.'

In [ ]:
data_train.iloc[0]['highlights']

'مدیرعامل شرکت ملی نفت، عملکرد مدیریت امور بین الملل این شرکت را در دوران تحریم بسیار هوشمندانه خواند و گفت: امور بین الملل در دوران پس از تحریم ها نیز می تواند نقش بزرگی در تسریع روند توسعه داشته باشد.'

In [ ]:
data_test.iloc[14]['article']

'به گزارش ایمنا به نقل از پایگاه اطلاع رسانی وزارت امور خارجه، سعید خطیب زاده دراین باره اظهار کرد: جمهوری اسلامی ایران ضمن ابراز تاسف از نقض آتش بس اعلام شده در درگیری های اخیر میان جمهوری آذربایجان و جمهوری ارمنستان و دعوت از طرفین به خویشتنداری بیشتر، حملات موشکی به زیر ساخت های حیاتی و مناطق مسکونی شهرها و کشتار غیر نظامیان را محکوم کرده و با خانواده های داغدار ابراز همدردی می کند. وی افزود: جمهوری اسلامی ایران مجددا از طرفین می خواهد ضمن پایبندی به آتش بس، گفت وگوهای خود را در چارچوب حقوق بین الملل و احترام به تمامیت ارضی یکدیگر و تخلیه شهرهای اشغالی از سر گیرند و در این راه آمادگی خود را برای تسهیل این گفت وگوها برای رسیدن به صلح و راه حل دا می و پایدار در منطقه اعلام می دارد.'

In [ ]:
summarize(data_test.iloc[14]['article'])

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1777: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


'زاده دراین باره گفت: جمهوری اسلامی ایران ضمن ابراز تاسف از نقض آتش بس اعلام شده در درگیری های اخیر میان جمهوری آذربایجان و جمهوری ارمنستان و دعوت از طرفین به خویشتنداری بیشتر، حملات موشکی به زیر ساخت های حیاتی و مناطق مسکونی شهرها و کشتار غیر نظامیان را محکوم کرده و با خانواده های داغدار ابراز همدردی می کند.'

In [ ]:
data_test.iloc[14]['highlights']

'سخنگوی وزارت امورخارجه در واکنش به نقض آتش بس اعلام شده میان جمهوری آذربایجان و ارمنستان گفت که ایران مجددا از دو طرف می خواهد ضمن پایبندی به آتش بس، گفت وگوهای خود را در چارچوب حقوق بین الملل و احترام به تمامیت ارضی یکدیگر و تخلیه شهرهای اشغالی از سر گیرند.'

In [ ]:
! pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rouge import Rouge

In [ ]:
rouge = Rouge()

In [ ]:
rouge.get_scores(summarize(data_test.iloc[14]['article']), data_test.iloc[14]['highlights'])

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1777: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


[{'rouge-1': {'f': 0.36559139286391495,
   'p': 0.3469387755102041,
   'r': 0.38636363636363635},
  'rouge-2': {'f': 0.12727272229917375,
   'p': 0.11864406779661017,
   'r': 0.13725490196078433},
  'rouge-l': {'f': 0.25806451114348483,
   'p': 0.24489795918367346,
   'r': 0.2727272727272727}}]